# Парсинг топ-250 фильмов IMDb

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [9]:
url = "https://ru.wikipedia.org/wiki/250_лучших_фильмов_по_версии_IMDb"

## Проверка успешности запроса


In [10]:
response = requests.get(url)
if response.status_code != 200:
    exit()

## Парсинг

In [11]:

soup = BeautifulSoup(response.text, "html.parser")
movies_inf = soup.find_all("tr")[1:]


## Извлечение данных из таблицы

Проходим по каждой строке таблицы и извлекаем информацию о фильмах в словарь.

In [12]:

movies_data = []
for row in movies_inf:
    cells = row.find_all("td")
    if len(cells) >= 5:
        movie = {
            'Место': cells[0].text.strip(),
            'Название': cells[1].text.strip(),
            'Год': cells[2].text.strip(),
            'Режиссёр': cells[3].text.strip(),
            'Рейтинг': cells[4].text.strip()
        }
        movies_data.append(movie)

## Создание датасета


In [13]:
df = pd.DataFrame(movies_data)

df.set_index('Место', inplace=True)

## Вывод результатов


In [16]:
df.head(40)

,Название,Год,Режиссёр,Рейтинг
Место,,,,
1,Побег из Шоушенка,1994,Фрэнк Дарабонт,"эпический фильм, историческая драма, тюремная ..."
2,Крёстный отец,1972,Фрэнсис Форд Коппола,"эпический фильм, гангстерский фильм, фильм-тра..."
3,Тёмный рыцарь,2008,Кристофер Нолан,"эпический боевик, эпический фильм, супергеройс..."
4,Крёстный отец 2,1974,Фрэнсис Форд Коппола,"эпический фильм, гангстерский фильм, фильм-тра..."
5,12 разгневанных мужчин,1957,Сидни Люмет,"юридическая драма, психологическая драма, дете..."
6,Властелин колец: Возвращение короля,2003,Питер Джексон,"приключенческая киноэпопея, эпический фильм, ф..."
7,Список Шиндлера,1993,Стивен Спилберг,"докудрама, эпический фильм, историческая киноэ..."
8,Криминальное чтиво,1994,Квентин Тарантино,"чёрная комедия, фильм о наркопреступлении, ган..."
9,Властелин колец: Братство Кольца,2001,Питер Джексон,"приключенческая киноэпопея, эпический фильм, ф..."
